# DATA CLEANING PART 2
## was going to do EDA but realised I needed to clean more goddammit

steps to take:
* calculate price per unit for every product
* get frequency of brands(how often they appear) per sub category
* average price per sub category
* compare by country price per unit by sub category
* what are the most frequent brands per sub category per country
* words used in product name
* words used in description(malaysia and philippines only)

Also need to reorder columns just gonna do it in excel tbh

In [1]:
import pandas as pd
import numpy as np #I barely have use for numpy yet I import it every time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re

price per unit calculation
* convert kg and l to g and ml by multiplying by a thousand
* replace kg with g replace l with ml(replace mll with ml)
* ppu formula = price / (quantity * amount)
* if units = g or ml then calculate cost per 10g or 10ml (ppu formula * 10)

### ok so was looking back at my data one last time before I do it and my amount/quantity regex wasn't perfect so there was some stuff I had to fix, regex is hard and I need to git gud

### the problem was diapers - the diapers had quantities in the name but no units just plain numbers so when I made the regex I made it look specifically for numbers and units 

### diaper quantities didn't get extracted properly as a result 

### but there was stuff where numbers didn't mean quantity so argreiurglwurgw

### anyway solved by using regex function in google sheets 

### I wish there was a regex function in excel(without using vba) like google sheets 

## More specific steps:

### 1. read the files
### 2. make columns equal
### 3. combine them
### 4. do EDA all at once

One annoying thing about using pandas is that it's kinda hard(right now, may need to just git gud) to literally see how things are. Kinda should use pandas more but sometimes excel is too useful. Maybe for much much larger datasets I can afford to just mass impute values, mass drop them etc.

In [152]:
raw_data = pd.read_excel("countries_combined.xlsx")

In [153]:
raw_data.head()

,name,price,cat1,cat2,cat3,brand,units,qty,amounts,ppu,special_requirement,country
0,Heinz Baby Biscotti (60g) - 4 Variants,6.9,Baby & Toys,Formula & Food,Puree,Heinz,g,1,60.0,None,-,MY
1,Snow Neo Baby Step 2 (900g),87.6,Baby & Toys,Formula & Food,Follow-On Milk (6-12 months),Snow,g,1,900.0,None,-,MY
2,Cuddly Plush Teddy Bear Cream with Cap 30cm,59.9,Baby & Toys,Toys & Education,Stuffed Toys,Cuddly Plush,ea,1,1.0,None,-,MY
3,Johnson's Baby Oil Lite (125ml),9.6,Baby & Toys,Kids Health & Skincare,Moisturizers,Johnson's,ml,1,125.0,None,-,MY
4,Carrie Junior Hair & Body Wash - Cheeky Cherry...,8.5,Baby & Toys,Bath & Toiletries,Soaps,Carrie,ml,1,280.0,None,-,MY


In [154]:
copy = raw_data.copy()

In [155]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57556 entries, 0 to 57555
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 57556 non-null  object 
 1   price                57556 non-null  object 
 2   cat1                 57556 non-null  object 
 3   cat2                 57556 non-null  object 
 4   cat3                 57556 non-null  object 
 5   brand                57539 non-null  object 
 6   units                57556 non-null  object 
 7   qty                  57556 non-null  int64  
 8   amounts              57556 non-null  float64
 9   ppu                  57556 non-null  object 
 10  special_requirement  57556 non-null  object 
 11  country              57556 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 5.3+ MB


In [156]:
units = set([i for i in copy["units"]])
units

{'cm',
 'ct',
 'ea',
 'g',
 'kg',
 'l',
 'ml',
 'mm',
 'pack',
 'pcs',
 's',
 'set',
 'sqm',
 '개',
 '매',
 '세트',
 '장',
 '조각',
 '종',
 '통',
 '팩'}

In [157]:
copy["qty"] = copy["qty"].astype(float)
copy["amounts"] = copy["amounts"].astype(float)

In [158]:
column = ["price"]
#removing price values that are strings eg: sold out
copy = (copy
       .drop(column,axis = 1)
       .join(copy[column].apply(pd.to_numeric,errors='coerce')))
copy[copy[column].notnull().all(axis=1)]

,name,cat1,cat2,cat3,brand,units,qty,amounts,ppu,special_requirement,country,price
0,Heinz Baby Biscotti (60g) - 4 Variants,Baby & Toys,Formula & Food,Puree,Heinz,g,1.0,60.0,None,-,MY,6.9
1,Snow Neo Baby Step 2 (900g),Baby & Toys,Formula & Food,Follow-On Milk (6-12 months),Snow,g,1.0,900.0,None,-,MY,87.6
2,Cuddly Plush Teddy Bear Cream with Cap 30cm,Baby & Toys,Toys & Education,Stuffed Toys,Cuddly Plush,ea,1.0,1.0,None,-,MY,59.9
3,Johnson's Baby Oil Lite (125ml),Baby & Toys,Kids Health & Skincare,Moisturizers,Johnson's,ml,1.0,125.0,None,-,MY,9.6
4,Carrie Junior Hair & Body Wash - Cheeky Cherry...,Baby & Toys,Bath & Toiletries,Soaps,Carrie,ml,1.0,280.0,None,-,MY,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...
57550,중모토이플러스 해피 330 휴대형 유모차,"Toys, Kids & Babies",휴대용유모차,-,중모토이플러스,ea,1.0,1.0,None,-,KR,48200.0
57552,GB 히어로 휴대형 유모차,"Toys, Kids & Babies",휴대용유모차,-,None,ea,1.0,1.0,None,-,KR,234380.0
57553,치코 C6 휴대형 유모차,"Toys, Kids & Babies",휴대용유모차,-,치코,ea,1.0,1.0,None,-,KR,89010.0
57554,톰비 GTX 휴대형 유모차,"Toys, Kids & Babies",휴대용유모차,-,톰비,ea,1.0,1.0,None,-,KR,69797.0


In [159]:
cols = list(copy.columns.values)
cols.insert(1,"price")
cols = cols[:-1]
cols
#copy = copy[cols]

['name',
 'price',
 'cat1',
 'cat2',
 'cat3',
 'brand',
 'units',
 'qty',
 'amounts',
 'ppu',
 'special_requirement',
 'country']

In [160]:
copy = copy[cols]

In [161]:
copy.head()

,name,price,cat1,cat2,cat3,brand,units,qty,amounts,ppu,special_requirement,country
0,Heinz Baby Biscotti (60g) - 4 Variants,6.9,Baby & Toys,Formula & Food,Puree,Heinz,g,1.0,60.0,None,-,MY
1,Snow Neo Baby Step 2 (900g),87.6,Baby & Toys,Formula & Food,Follow-On Milk (6-12 months),Snow,g,1.0,900.0,None,-,MY
2,Cuddly Plush Teddy Bear Cream with Cap 30cm,59.9,Baby & Toys,Toys & Education,Stuffed Toys,Cuddly Plush,ea,1.0,1.0,None,-,MY
3,Johnson's Baby Oil Lite (125ml),9.6,Baby & Toys,Kids Health & Skincare,Moisturizers,Johnson's,ml,1.0,125.0,None,-,MY
4,Carrie Junior Hair & Body Wash - Cheeky Cherry...,8.5,Baby & Toys,Bath & Toiletries,Soaps,Carrie,ml,1.0,280.0,None,-,MY


In [162]:
copy.reset_index(inplace=True)

In [163]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57556 entries, 0 to 57555
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                57556 non-null  int64  
 1   name                 57556 non-null  object 
 2   price                56325 non-null  float64
 3   cat1                 57556 non-null  object 
 4   cat2                 57556 non-null  object 
 5   cat3                 57556 non-null  object 
 6   brand                57539 non-null  object 
 7   units                57556 non-null  object 
 8   qty                  57556 non-null  float64
 9   amounts              57556 non-null  float64
 10  ppu                  57556 non-null  object 
 11  special_requirement  57556 non-null  object 
 12  country              57556 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 5.7+ MB


In [164]:
copy = copy.dropna()
copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56308 entries, 0 to 57555
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                56308 non-null  int64  
 1   name                 56308 non-null  object 
 2   price                56308 non-null  float64
 3   cat1                 56308 non-null  object 
 4   cat2                 56308 non-null  object 
 5   cat3                 56308 non-null  object 
 6   brand                56308 non-null  object 
 7   units                56308 non-null  object 
 8   qty                  56308 non-null  float64
 9   amounts              56308 non-null  float64
 10  ppu                  56308 non-null  object 
 11  special_requirement  56308 non-null  object 
 12  country              56308 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 6.0+ MB


In [165]:
copy.reset_index(inplace=True)
copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56308 entries, 0 to 56307
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   level_0              56308 non-null  int64  
 1   index                56308 non-null  int64  
 2   name                 56308 non-null  object 
 3   price                56308 non-null  float64
 4   cat1                 56308 non-null  object 
 5   cat2                 56308 non-null  object 
 6   cat3                 56308 non-null  object 
 7   brand                56308 non-null  object 
 8   units                56308 non-null  object 
 9   qty                  56308 non-null  float64
 10  amounts              56308 non-null  float64
 11  ppu                  56308 non-null  object 
 12  special_requirement  56308 non-null  object 
 13  country              56308 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 6.0+ MB


In [166]:
false_index = list(copy.columns.values)[:2]
false_index

['level_0', 'index']

In [167]:
copy = copy.drop(false_index,axis=1)
copy.head()

,name,price,cat1,cat2,cat3,brand,units,qty,amounts,ppu,special_requirement,country
0,Heinz Baby Biscotti (60g) - 4 Variants,6.9,Baby & Toys,Formula & Food,Puree,Heinz,g,1.0,60.0,None,-,MY
1,Snow Neo Baby Step 2 (900g),87.6,Baby & Toys,Formula & Food,Follow-On Milk (6-12 months),Snow,g,1.0,900.0,None,-,MY
2,Cuddly Plush Teddy Bear Cream with Cap 30cm,59.9,Baby & Toys,Toys & Education,Stuffed Toys,Cuddly Plush,ea,1.0,1.0,None,-,MY
3,Johnson's Baby Oil Lite (125ml),9.6,Baby & Toys,Kids Health & Skincare,Moisturizers,Johnson's,ml,1.0,125.0,None,-,MY
4,Carrie Junior Hair & Body Wash - Cheeky Cherry...,8.5,Baby & Toys,Bath & Toiletries,Soaps,Carrie,ml,1.0,280.0,None,-,MY


In [168]:
copy["country"] = copy["country"].apply(lambda x: x.replace("IN","ID"))

In [169]:
copy.to_excel("backup_countries_combined.xlsx",index=False)

In [170]:
copy = pd.read_excel("backup_countries_combined.xlsx")

In [45]:
ppu_list = []
for i in copy.index:
    ppu = copy["ppu"][i]
    units = copy["units"][i]
    price = copy["price"][i]
    qty = copy["qty"][i]
    amnts = copy["amounts"][i]
    if ppu == "None" and units in ["g","ml"]:
        ppu_list.append(price/(qty*amnts)*10)
    elif ppu == "None" and units in ["kg","l"]:
        ppu_list.append(price/(qty*amnts*1000)*10)
    elif ppu == "None":
        ppu_list.append(price/(qty*amnts))
    else:
        temp = ppu.split("/")
        extracted = float(re.match("\d+",temp[0]).group(0))
        ppu_list.append(extracted)
copy["ppu"] = ppu_list

ValueError: Length of values (0) does not match length of index (56308)

In [174]:
copy.to_excel("backup_countries_combined2.xlsx",index=False)

In [80]:
country = set([i for i in copy["country"]])
country

{'ID', 'KR', 'MY', 'PH', 'SG'}

### converting from other currencies to won

In [76]:
from forex_python.converter import CurrencyRates
c = CurrencyRates()

In [83]:
c.get_rates('IDR')["KRW"]

0.0783350313

#### curency codes are
##### Korea = KRW
##### Singapore = SGD
##### Malaysia = MYR
##### Indonesia = IDR
##### Philippines = PHP

In [84]:
country_codes = ["SGD","MYR","IDR","PHP"]
exchange_rates = {}
for i in country_codes:
    exchange_rates[i] = c.get_rates(i)["KRW"]
exchange_rates

{'SGD': 831.1709253287,
 'MYR': 272.6769618914,
 'IDR': 0.0783350313,
 'PHP': 22.9376829206}

In [175]:
exchange_rates

{'SGD': 831.1709253287,
 'MYR': 272.6769618914,
 'IDR': 0.0783350313,
 'PHP': 22.9376829206}

In [86]:
type(exchange_rates['SGD'])

float

In [176]:
ppu_converted = []
for i in copy.index:
    country = copy["country"][i]
    ppu = copy["ppu"][i]
    if country == "KR":
        ppu_converted.append(ppu)
    elif country == "SG":
        converted = ppu * exchange_rates['SGD']
        ppu_converted.append(converted)
    elif country == "MY":
        converted = ppu * exchange_rates['MYR']
        ppu_converted.append(converted)
    elif country == "ID":
        converted = ppu * exchange_rates['IDR']
        ppu_converted.append(converted)
    elif country == "PH":
        converted = ppu * exchange_rates['PHP']
        ppu_converted.append(converted)
copy["ppu"] = ppu_converted

In [177]:
copy.head()

,name,price,cat1,cat2,cat3,brand,units,qty,amounts,ppu,special_requirement,country
0,Heinz Baby Biscotti (60g) - 4 Variants,6.9,Baby & Toys,Formula & Food,Puree,Heinz,g,1,60.0,313.578506,-,MY
1,Snow Neo Baby Step 2 (900g),87.6,Baby & Toys,Formula & Food,Follow-On Milk (6-12 months),Snow,g,1,900.0,265.405576,-,MY
2,Cuddly Plush Teddy Bear Cream with Cap 30cm,59.9,Baby & Toys,Toys & Education,Stuffed Toys,Cuddly Plush,ea,1,1.0,16333.350017,-,MY
3,Johnson's Baby Oil Lite (125ml),9.6,Baby & Toys,Kids Health & Skincare,Moisturizers,Johnson's,ml,1,125.0,209.415907,-,MY
4,Carrie Junior Hair & Body Wash - Cheeky Cherry...,8.5,Baby & Toys,Bath & Toiletries,Soaps,Carrie,ml,1,280.0,82.776935,-,MY


In [178]:
copy.to_excel("backup_countries_combined3.xlsx",index=False)

### I keep doing dumb shit and wasting time

### categories were all over the place for the countries so looked over them and unified the categories

In [33]:
raw = pd.read_excel("backup_countries_combined3.xlsx",sheet_name="Sheet1")
categories_fixed = pd.read_excel("backup_countries_combined3.xlsx",sheet_name="Sheet2")

In [34]:
categories_fixed.head(20)

,cat2,cat2mapped
0,Formula,Formula
1,Food,Food
2,Toys & Education,Toys & Education
3,Kids Health & Skincare,Kids Health & Skincare
4,Bath & Toiletries,Bath & Toiletries
5,Diapers & Potties,Diapers
6,Nursery,Nursery
7,Feeding & Nursing,Feeding & Nursing
8,Baby Gear,Baby Gear
9,Baby & Toddler Play,Baby & Toddler Play


In [35]:
category_map = categories_fixed.set_index('cat2').T.to_dict('list')

In [36]:
copy = raw.copy()

In [37]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56308 entries, 0 to 56307
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 56308 non-null  object 
 1   price                56308 non-null  float64
 2   cat1                 56308 non-null  object 
 3   cat2                 56308 non-null  object 
 4   cat3                 56308 non-null  object 
 5   brand                56308 non-null  object 
 6   units                56308 non-null  object 
 7   qty                  56308 non-null  int64  
 8   amounts              56308 non-null  float64
 9   ppu                  56308 non-null  float64
 10  special_requirement  56308 non-null  object 
 11  country              56308 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 5.2+ MB


In [38]:
category_map[copy["cat2"][0]][0]

'Food'

In [39]:
fixed_cat2 = []

for i in copy.index:
    fixed = category_map[copy["cat2"][i]][0]
    fixed_cat2.append(fixed)

copy["cat2"] = fixed_cat2

In [40]:
copy.to_excel("backup_countries_combined4.xlsx",index=False)